In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from evoman.environment import Environment
from controller_custom import player_controller
from multiprocessing import Pool
from scipy.stats import ttest_ind, mannwhitneyu
import pandas as pd
from deap import base, creator, tools, algorithms
import warnings

pygame 2.6.0 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

In [3]:
def run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group):
    """
    Runs Genetic Algorithm B with Crowding for preserving diversity using DEAP.

    Parameters:
    - env: Evoman Environment instance.
    - npopulation: Population size.
    - gens: Number of generations.
    - mutation_rate: Mutation rate.
    - dom_u: Upper bound for gene values.
    - dom_l: Lower bound for gene values.
    - enemy_group: Enemy group identifier.

    Returns:
    - history_mean: List of mean fitness per generation.
    - history_max: List of max fitness per generation.
    - best_solution: Best solution found.
    """

    # Define the problem as a maximization problem
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

    n_hidden_neurons = 10
    n_vars = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5

    # Evaluation function (simulating the environment)
    def evaluate(individual):
        env.player_controller.set(individual, env.get_num_sensors())
        fitness, _, _, _ = env.play(pcont=individual)
        return fitness,

    # Setup DEAP toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_float", np.random.uniform, dom_l, dom_u)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=n_vars)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxUniform, indpb=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=mutation_rate)
    toolbox.register("select", tools.selTournament, tournsize=5)

    # Initialize population
    population = toolbox.population(n=npopulation)

    # Statistics to record the performance of the algorithm
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Evaluate the entire population initially
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    # Record fitness over generations
    history_mean = []
    history_max = []

    # Crowding distance calculation (Euclidean distance between two individuals)
    def crowding_distance(ind1, ind2):
        return np.linalg.norm(ind1 - ind2)

    # Custom crowding replacement logic
    def crowding_replacement(parent1, parent2, child1, child2):
        # Compute the Euclidean distances
        d_p1_o1 = crowding_distance(parent1, child1)
        d_p1_o2 = crowding_distance(parent1, child2)
        d_p2_o1 = crowding_distance(parent2, child1)
        d_p2_o2 = crowding_distance(parent2, child2)

        # Assign children based on the minimum distance to parents
        if d_p1_o1 + d_p2_o2 < d_p1_o2 + d_p2_o1:
            # Pair child1 with parent1 and child2 with parent2
            if child1.fitness.values[0] > parent1.fitness.values[0]:
                parent1[:] = child1
                parent1.fitness.values = child1.fitness.values
            if child2.fitness.values[0] > parent2.fitness.values[0]:
                parent2[:] = child2
                parent2.fitness.values = child2.fitness.values
        else:
            # Pair child1 with parent2 and child2 with parent1
            if child1.fitness.values[0] > parent2.fitness.values[0]:
                parent2[:] = child1
                parent2.fitness.values = child1.fitness.values
            if child2.fitness.values[0] > parent1.fitness.values[0]:
                parent1[:] = child2
                parent1.fitness.values = child2.fitness.values

    # Genetic Algorithm Loop
    for generation in range(gens):
        # Select offspring using tournament selection
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for i in range(0, len(offspring), 2):
            parent1, parent2 = offspring[i], offspring[i + 1]

            # Crossover and mutate
            if np.random.rand() < 0.5:
                toolbox.mate(parent1, parent2)

            toolbox.mutate(parent1)
            toolbox.mutate(parent2)
            del parent1.fitness.values
            del parent2.fitness.values

            # Evaluate the fitness of the children
            parent1.fitness.values = toolbox.evaluate(parent1)
            parent2.fitness.values = toolbox.evaluate(parent2)

            # Apply crowding replacement
            crowding_replacement(population[i], population[i + 1], parent1, parent2)

        # Gather statistics for this generation
        record = stats.compile(population)
        history_mean.append(record["avg"])
        history_max.append(record["max"])

        # Logging
        print(f'Generation {generation}: Best Fitness = {history_max[-1]:.4f}, Mean Fitness = {history_mean[-1]:.4f}, Std = {record["std"]:.4f}')

    # Get best solution
    best_individual = tools.selBest(population, 1)[0]

    # Save the best solution to a file
    best_solution = np.array(best_individual)
    np.savetxt(os.path.join('results', f'best_solution_B_{enemy_group}.txt'), best_solution)

    return history_mean, history_max, best_solution


## Parameters

In [4]:
# Experiment Parameters

enemy_group_1 = [1, 4, 5, 8]
enemy_group_2 = [2, 3, 6, 7]

num_runs = 2 			# Set to 10
npopulation = 200
gens = 10 				# Set to 30
mutation_rate = 0.05
dom_u, dom_l = 1, -1
elitism_count_a = 1
elitism_count_b = 2
 

# The number of hidden neurons for the player controller
n_hidden_neurons = 10

# Initialize data storage
results = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Generation': [],
    'Mean Fitness': [],
    'Max Fitness': [],
    'Best Solution': []
}

best_solutions = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Best Solution': []
}

# Run Algorithm B on Enemy Group 1

In [5]:
print('Running Algorithm B for enemy group 1')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA2_EG1'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_1,
						playermode="ai",
						multiplemode="yes",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_1')

	for gen in range(gens):
		results['Algorithm'].append('B')
		results['Enemy Group'].append(1)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('B')
	best_solutions['Enemy Group'].append(1)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA2_EG12.csv'), index=False)

Running Algorithm B for enemy group 1
Run 1

MESSAGE: Pygame initialized for simulation.
Generation 0: Best Fitness = 30.6457, Mean Fitness = -0.7327, Std = 8.6416
Generation 1: Best Fitness = 42.0686, Mean Fitness = 7.9349, Std = 10.6808
Generation 2: Best Fitness = 46.2255, Mean Fitness = 15.3615, Std = 10.6602
Generation 3: Best Fitness = 60.7371, Mean Fitness = 21.8054, Std = 11.3135
Generation 4: Best Fitness = 60.7371, Mean Fitness = 27.9257, Std = 10.9071
Generation 5: Best Fitness = 60.7371, Mean Fitness = 32.9615, Std = 11.0492
Generation 6: Best Fitness = 60.7371, Mean Fitness = 38.5265, Std = 11.0573
Generation 7: Best Fitness = 67.1773, Mean Fitness = 44.2220, Std = 9.9236
Generation 8: Best Fitness = 67.4549, Mean Fitness = 47.3762, Std = 9.4034
Generation 9: Best Fitness = 69.2050, Mean Fitness = 50.5943, Std = 8.9850
Run 2

MESSAGE: Pygame initialized for simulation.
Generation 0: Best Fitness = 46.6278, Mean Fitness = -0.8646, Std = 10.1969
Generation 1: Best Fitness = 